In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('sentiwordnet')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:


import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
df = pd.read_csv("headphone.csv")

# Check column names
df.columns


Index(['ReviewBody', 'ReviewStar'], dtype='object')

In [ ]:
stop_words = set(stopwords.words('english'))

negations = {'not', 'no', 'never', "n't"}
stop_words = stop_words - negations

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    return tokens

df['tokens'] = df['ReviewBody'].apply(clean_text)
df.head()


,ReviewBody,ReviewStar,tokens
0,No doubt it has a great bass and to a great ex...,3,"[no, doubt, great, bass, great, extent, noise,..."
1,"This earphones are unreliable, i bought it be...",1,"[earphones, unreliable, bought, days, meanwhil..."
2,"i bought itfor 999,I purchased it second time,...",4,"[bought, itfor, purchased, second, time, gifte..."
3,Its sound quality is adorable. overall it was ...,1,"[sound, quality, adorable, overall, good, week..."
4,Its Awesome... Good sound quality & 8-9 hrs ba...,5,"[awesome, good, sound, quality, hrs, battery, ..."


def rake_phrases(tokens):
    phrases = []
    phrase = []
    for word in tokens:
        if word in stop_words:
            if phrase:
                phrases.append(phrase)
                phrase = []
        else:
            phrase.append(word)
    if phrase:
        phrases.append(phrase)
    return phrases


word_freq = {}
word_degree = {}

for tokens in df['tokens']:
    phrases = rake_phrases(tokens)
    for phrase in phrases:
        degree = len(phrase)
        for word in phrase:
            word_freq[word] = word_freq.get(word, 0) + 1
            word_degree[word] = word_degree.get(word, 0) + degree

word_score = {w: word_degree[w] / word_freq[w] for w in word_freq}

def phrase_score(phrase):
    return sum(word_score[w] for w in phrase)


In [ ]:
def rake_phrases(tokens):
    phrases = []
    phrase = []
    for word in tokens:
        if word in stop_words:
            if phrase:
                phrases.append(phrase)
                phrase = []
        else:
            phrase.append(word)
    if phrase:
        phrases.append(phrase)
    return phrases


word_freq = {}
word_degree = {}

for tokens in df['tokens']:
    phrases = rake_phrases(tokens)
    for phrase in phrases:
        degree = len(phrase)
        for word in phrase:
            word_freq[word] = word_freq.get(word, 0) + 1
            word_degree[word] = word_degree.get(word, 0) + degree

word_score = {w: word_degree[w] / word_freq[w] for w in word_freq}

def phrase_score(phrase):
    return sum(word_score[w] for w in phrase)

aspect_candidates = []

for tokens in df['tokens']:
    phrases = rake_phrases(tokens)
    for p in phrases:
        if len(p) > 1:
            aspect_candidates.append((" ".join(p), phrase_score(p)))

aspects_df = pd.DataFrame(aspect_candidates, columns=['aspect', 'score'])

top_aspects = (
    aspects_df
    .groupby('aspect')
    .mean()
    .sort_values('score', ascending=False)
    .head(15)
)

top_aspects

,score
aspect,
update sadly second boat product conk months use guess end journey boat audio not buying products brand changed rating reflect samei market cheap pair bluetooth earphones chanced upon lightning deal bought used boat products good build quality sound alright im writing month enough time lived get know quirks problems sound nothing great bit bassy side youre bass lover definitely good buy connectivity quick painless works without hassle every time quality connectivity depends sometimes weird put phone charge use distance meters no worries sometimes im walking really fast jogging breaks crackles audio not sure material gym wear works well phones jeans battery life nothing write home listen music constantly even im work demands probably high price range sweatproof advertised no interruptions light drizzle got great build quality price looks feels premium doesnt tangle stays ear regardless intensity exercise greaton whole great buy looking cheaper alternative wait sale get thoughlike reviews please click helpful button encourages share products use thanks,7788.519668
build well designed lightweight comfortable wires good not flimsy wearing comfort well maintained better mi bluetooth earphones main competitor boat price categorycomfort mentioned earphones really comfortable fits ear no chance slipping unless youre trying desperately internationally theyre lightweight no hazard department perfect long hours music playbacksound important aspect earphone sound clarity boat surely doesnt disappoint well bit base heavy sound clear crisp loud good noise cancellation also helps isolate music playback although judging pure audiophiles point view mi bluetooth earphones might balanced sound signature boat bit base heavy awesome hiphops bollywood genres music player good equalizer preferably poweramp android going elevate listening experience even morebattery single charge gives hours music playback decent enough full charge takes roughly hoursverdict nutshell boat rockerz worth price tag better build quality comfort bluetooth range packs punch competition extreme purists sound signature might bit turn people even people good ears going great experience,7665.737702
never used boat product one boy impressed quality thingbass mids highs battery performance hours full charge also feature mins charge give mins worth batterybt range depending upon obstacles phonecomfort plasticysilicone actually things protruding might feel bit uncomfortable firstcons irritating indicator low battery presume goes timesat first every drop percentage hearing oops battery getting low similar message charging port concealed awkard wayalthough durable cover kinda obtrusive recommend not using powerfastturbo charger whatever call days want see live longer monthmah battery lion battery would def not like charge mine using laptop usb portsometimes charged using phone drain full charge epic travellingi used many wired earphones comparison seems stupid well competing sennheiser cx jbl csi even outperforms sony mdrexap terms sound qualityalso getting around seems like best price anything higher would save sumtin elsetldr fits budget go without thinking,7435.272309
product great intially real test months useso reviewing product two months usepros earphones drives rich sound quality inspite wireless neck band makes much durable earphones flat wire durable magnets saves loosing degree tilted earbuds fits right ears check degree tilted earphones day weather comfortable not comes pairs extra buds small big medium one already itcons connectivity dual phones not easy shownconnect earphone phone usualdisconnect phone switching bluetooth phone anow connect earphone phone bswitch earphone switch backopen bluetooth settings phone bfrom paired device list select boat rockers device timeon earphones get hear connected boat rockerz twice get cracking sound ideal dial number dial pad battery life not accuratesometimes work hours battery sometime dead dont rely battery indicat

In [ ]:
def adjective_sentiment(word):
    synsets = list(swn.senti_synsets(word, 'a'))
    if not synsets:
        return 0
    pos = np.mean([s.pos_score() for s in synsets])
    neg = np.mean([s.neg_score() for s in synsets])
    return pos - neg


In [ ]:
def sentiment_with_negation(tokens):
    sentiment = 0
    for i, word in enumerate(tokens):
        score = adjective_sentiment(word)
        if i > 0 and tokens[i-1] in negations:
            score *= -1
        sentiment += score
    return sentiment


In [ ]:
aspect_sentiments = []

for aspect in top_aspects.index:
    aspect_words = aspect.split()
    for tokens in df['tokens']:
        if all(w in tokens for w in aspect_words):
            score = sentiment_with_negation(tokens)
            aspect_sentiments.append((aspect, score))

aspect_df = pd.DataFrame(aspect_sentiments, columns=['aspect', 'sentiment'])

final_summary = (
    aspect_df
    .groupby('aspect')
    .agg(
        frequency=('sentiment', 'count'),
        avg_sentiment=('sentiment', 'mean')
    )
    .sort_values('avg_sentiment', ascending=False)
)

final_summary


,frequency,avg_sentiment
aspect,,
build well designed lightweight comfortable wires good not flimsy wearing comfort well maintained better mi bluetooth earphones main competitor boat price categorycomfort mentioned earphones really comfortable fits ear no chance slipping unless youre trying desperately internationally theyre lightweight no hazard department perfect long hours music playbacksound important aspect earphone sound clarity boat surely doesnt disappoint well bit base heavy sound clear crisp loud good noise cancellation also helps isolate music playback although judging pure audiophiles point view mi bluetooth earphones might balanced sound signature boat bit base heavy awesome hiphops bollywood genres music player good equalizer preferably poweramp android going elevate listening experience even morebattery single charge gives hours music playback decent enough full charge takes roughly hoursverdict nutshell boat rockerz worth price tag better build quality comfort bluetooth range packs punch competition extreme purists sound signature might bit turn people even people good ears going great experience,1,8.754853
using boat wireless headphones since two years never disappointed methis rs boat wireless headphones must say one impressed alot according price segment rs perfect job delivering good quality soundregarding battery life hrs quite ok since purchased rs cant expect much itcoming sound quality bit bass bass treble not problem love high bassoverall sound quality good price segmentthe build quality not high quality yes sturdy enough withstand normal dropsacording length cord lengthyone thing dint like boaat connectivity range range connectivity less mbut budget matters given thought paid rsi much satisfied overall performanceregarding service center customers leaving mumbai added advantage hadmy previous boat rockerz one earpieces suddenly stopped visited service center goregaon replaced headphones new minutes without questions askedso customers mumbai dont worry warranty,1,7.442370
doesnt get know realistic review productive using days understood sound quality fukin love earphonesnoise cancellation good cant hear noises try hear themcomfortability cant listen longer period time start paining mins not comfortable removed wings old eartips using right comfortable listen hours straight without painingbattery life excellent used days battery percentage dropped sound qualitymids average wont miss soundhighs treble vocals not mark adjust equaliser settings make sounds amazinglows bass powerful rockerz made bass lovers guess focus bass earphones extra bassi saying many times literally much bassthe people saying would better bass stupid funi used high mid end earphones like sony mdr xb extra bass jbl oneplus bulletsat first heard thought made mistake purchasing earphones equalised made sounds like high end earphones happy purchase nowyou like didnt high end earphones never heard high end earphones wants eargasm must say go product youll happy purchase dont forget equalise,1,6.445635
update sadly second boat product conk months use guess end journey boat audio not buying products brand changed rating reflect samei market cheap pair bluetooth earphones chanced upon lightning deal bought used boat products good build quality sound alright im writing month enough time lived get know quirks problems sound nothing great bit bassy side youre bass lover definitely good buy connectivity quick painless works without hassle every time quality connectivity depends sometimes weird put phone charge use distance meters no worries sometimes im walking really fast jogging breaks crackles audio not sure material gym wear works well phones jeans battery life nothing write home listen music constantly even im work demands probably high price range sweatproof advertised no interruptions light drizzle got great build quality price looks feels premium doesnt tangle stays ear regardless intensity exercise greaton whole great buy looking cheape

In [ ]:
low_rated = df[df['ReviewStar'] <= 3]
low_rated.head()


,ReviewBody,ReviewStar,tokens
0,No doubt it has a great bass and to a great ex...,3,"[no, doubt, great, bass, great, extent, noise,..."
1,"This earphones are unreliable, i bought it be...",1,"[earphones, unreliable, bought, days, meanwhil..."
3,Its sound quality is adorable. overall it was ...,1,"[sound, quality, adorable, overall, good, week..."
5,"After 11 days, the charging port isn't working...",1,"[days, charging, port, isnt, working, cant, ev..."
7,After one month usage reviewing this .Pros-1- ...,3,"[one, month, usage, reviewing, pros, sound, qu..."


In [ ]:

final_summary.to_csv("aspect_sentiment_results.csv")
